In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
sns.set()

In [2]:
df_main = pd.read_csv('climate_change_data/API_19_DS2_en_csv_v2_5361599.csv', sep=',', skiprows=4)
df_main = df_main.iloc[:, :-1]

id_columns = ['Country Name',
              'Country Code',
              'Indicator Name',
              'Indicator Code']
df_main = pd.melt(df_main, id_vars=id_columns,
                  var_name='Year',
                  value_name='Value')
df_main['Year'] = pd.to_numeric(df_main['Year'])
df_main

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,Urban population (% of total population),SP.URB.TOTL.IN.ZS,1960,50.776
1,Aruba,ABW,Urban population,SP.URB.TOTL,1960,27728.000
2,Aruba,ABW,Urban population growth (annual %),SP.URB.GROW,1960,NaN
3,Aruba,ABW,"Population, total",SP.POP.TOTL,1960,54608.000
4,Aruba,ABW,Population growth (annual %),SP.POP.GROW,1960,NaN
...,...,...,...,...,...,...
1253387,Zimbabwe,ZWE,Rural land area where elevation is below 5 met...,AG.LND.EL5M.RU.ZS,2021,NaN
1253388,Zimbabwe,ZWE,Rural land area where elevation is below 5 met...,AG.LND.EL5M.RU.K2,2021,NaN
1253389,Zimbabwe,ZWE,Arable land (% of land area),AG.LND.ARBL.ZS,2021,NaN
1253390,Zimbabwe,ZWE,Agricultural land (% of land area),AG.LND.AGRI.ZS,2021,NaN


In [3]:
meta_country_df = pd.read_csv('climate_change_data/Metadata_Country_API_19_DS2_en_csv_v2_5361599.csv')
meta_country_df = meta_country_df.iloc[:, :-1]

In [4]:
meta_indicators_df = pd.read_csv('climate_change_data/Metadata_Indicator_API_19_DS2_en_csv_v2_5361599.csv')
meta_indicators_df.columns = meta_indicators_df.columns.str.lower()
meta_indicators_df.sort_values(['indicator_code'], inplace=True)
meta_indicators_df = meta_indicators_df.reset_index(drop=True)
meta_indicators_df = meta_indicators_df.iloc[:, :-1]

In [5]:
mask = (df_main['Year'] >= 1990) & (df_main['Year'] <= 2012)
df_main = df_main[mask]
selected_metrics = ["Renewable electricity output (% of total electricity output)",
                    "Energy use (kg of oil equivalent per capita)",
                    "Electric power consumption (kWh per capita)",
                    "Electricity production from renewable sources, excluding hydroelectric (kWh)",
                    "Electricity production from coal sources (% of total)",
                    "Electricity production from hydroelectric sources (% of total)",
                    "Electricity production from natural gas sources (% of total)",
                    "Electricity production from nuclear sources (% of total)",
                    "Electricity production from oil sources (% of total)",
                    "Electricity production from renewable sources, excluding hydroelectric (% of total)",
                    "Population, total",
                    "Population growth (annual %)",
                    "Urban population (% of total population)",
                    "Urban population growth (annual %)",
                    "Foreign direct investment, net inflows (% of GDP)",
                    "Mortality rate, under-5 (per 1,000 live births)",
                    "Agricultural land (% of land area)",
                    "Forest area (% of land area)",
                    "Arable land (% of land area)",
                    "Agriculture, forestry, and fishing, value added (% of GDP)",
                    "Methane emissions (kt of CO2 equivalent)",
                    "Nitrous oxide emissions (thousand metric tons of CO2 equivalent)",
                    "Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)",
                    "CO2 emissions (metric tons per capita)",
                    "CO2 emissions from liquid fuel consumption (% of total)",
                    "CO2 emissions from solid fuel consumption (% of total)",
                    "CO2 emissions from gaseous fuel consumption (% of total)"]


df_main = df_main[df_main['Indicator Name'].isin(selected_metrics)]
meta_indicators_df = meta_indicators_df[meta_indicators_df['indicator_name'].isin(selected_metrics)]


# Переименовывание столбцоы
cols = ['country_name', 'country_code', 'indicator_name', 'indicator_code', 'year', 'value']
df_main.columns = cols
df_main

# Сводная таблицы
df_pivot = pd.pivot_table(df_main,
                          index=['country_name','country_code', 'year'],
                          columns='indicator_code',
                          values=['value'])
df_pivot = df_pivot.reset_index()
cols = list(df_pivot.columns.get_level_values(1))
cols[:3] = ['country_name', 'country_code', 'year']
df_pivot.columns = cols
df_pivot


# Добавление incomeGroup
income_data = meta_country_df[['Country Code', 'IncomeGroup']]
df_pivot = df_pivot.merge(income_data,
                          how='left',
                          left_on='country_code',
                          right_on='Country Code')
df_pivot.drop('Country Code', axis=1, inplace=True)


# Тольк страны

In [6]:
region_nan_mask = meta_country_df['Region'].isna()

aggregate_cols = meta_country_df.loc[region_nan_mask, 'TableName']
aggregate_cols

union_aggregate = {'Arab World',
                   'European Union'
                   'IBRD only',
                   'IDA & IBRD total',
                   'IBRD only',
                   'IDA total',
                   'IDA blend',
                   'IDA only',
                   'OECD members'}


poor_aggregate = {
    'Fragile and conflict affected situations',
    'Heavily indebted poor countries (HIPC)',
    'Least developed countries: UN classification'}

income_aggregate = {'Low income',
                    'Lower middle income',
                    'Low & middle income',
                    'Upper middle income',
                    'Middle income',
                    'High income'}

dividend_aggregate = {'Early-demographic dividend',
                      'Late-demographic dividend',
                      'Pre-demographic dividend',
                      'Post-demographic dividend'}

all_without_region = union_aggregate.union(poor_aggregate).union(income_aggregate).union(dividend_aggregate)


region_aggregate = set(aggregate_cols.to_list()).difference(all_without_region)
region_aggregate

{'Africa Eastern and Southern',
 'Africa Western and Central',
 'Caribbean small states',
 'Central Europe and the Baltics',
 'East Asia & Pacific',
 'East Asia & Pacific (IDA & IBRD)',
 'East Asia & Pacific (excluding high income)',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (IDA & IBRD)',
 'Europe & Central Asia (excluding high income)',
 'European Union',
 'Latin America & Caribbean',
 'Latin America & Caribbean (IDA & IBRD)',
 'Latin America & Caribbean (excluding high income)',
 'Middle East & North Africa',
 'Middle East & North Africa (IDA & IBRD)',
 'Middle East & North Africa (excluding high income)',
 'North America',
 'Other small states',
 'Pacific island small states',
 'Small states',
 'South Asia',
 'South Asia (IDA & IBRD)',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD)',
 'Sub-Saharan Africa (excluding high income)',
 'World'}

In [7]:
aggregate_cols

1                            Africa Eastern and Southern
3                             Africa Western and Central
7                                             Arab World
36                        Central Europe and the Baltics
49                                Caribbean small states
61           East Asia & Pacific (excluding high income)
62                            Early-demographic dividend
63                                   East Asia & Pacific
64         Europe & Central Asia (excluding high income)
65                                 Europe & Central Asia
68                                             Euro area
73                                        European Union
74              Fragile and conflict affected situations
95                                           High income
98                Heavily indebted poor countries (HIPC)
102                                            IBRD only
103                                     IDA & IBRD total
104                            

In [8]:
# убираем aggreagetive cols
df_pivot = df_pivot[~df_pivot['country_name'].isin(aggregate_cols)]

## GPD add

In [9]:
gdp_df = pd.read_csv('climate_change_data/GDP_data.csv', sep=',', skiprows=4)

id_columns = ['Country Name',
              'Country Code']
gdp_df = pd.melt(gdp_df, id_vars=id_columns,
                 var_name='Year',
                 value_name='Value')

numeric_year_mask = gdp_df['Year'].str.isnumeric()
gdp_df = gdp_df[numeric_year_mask]
gdp_df['Year'] = pd.to_numeric(gdp_df['Year'])

cols = ['country_name', 'country_code', 'year', 'gdp']
gdp_df.columns = cols
gdp_df

,country_name,country_code,year,gdp
532,Aruba,ABW,1960,NaN
533,Africa Eastern and Southern,AFE,1960,21291524631.3606
534,Afghanistan,AFG,1960,537777811.111111
535,Africa Western and Central,AFW,1960,10404135069.15
536,Angola,AGO,1960,NaN
...,...,...,...,...
17285,Kosovo,XKX,2022,NaN
17286,"Yemen, Rep.",YEM,2022,NaN
17287,South Africa,ZAF,2022,NaN
17288,Zambia,ZMB,2022,NaN


### merge gdp wit main

In [10]:
df_merged = df_pivot.merge(gdp_df[['year', 'country_code', 'gdp']], on=['year', 'country_code'], how='left')

In [11]:
df_merged.columns

Index(['country_name', 'country_code', 'year', 'AG.LND.AGRI.ZS',
       'AG.LND.ARBL.ZS', 'AG.LND.FRST.ZS', 'BX.KLT.DINV.WD.GD.ZS',
       'EG.ELC.COAL.ZS', 'EG.ELC.HYRO.ZS', 'EG.ELC.NGAS.ZS', 'EG.ELC.NUCL.ZS',
       'EG.ELC.PETR.ZS', 'EG.ELC.RNEW.ZS', 'EG.ELC.RNWX.KH', 'EG.ELC.RNWX.ZS',
       'EG.USE.ELEC.KH.PC', 'EG.USE.PCAP.KG.OE', 'EN.ATM.CO2E.GF.ZS',
       'EN.ATM.CO2E.LF.ZS', 'EN.ATM.CO2E.PC', 'EN.ATM.CO2E.SF.ZS',
       'EN.ATM.GHGO.KT.CE', 'EN.ATM.METH.KT.CE', 'EN.ATM.NOXE.KT.CE',
       'NV.AGR.TOTL.ZS', 'SH.DYN.MORT', 'SP.POP.GROW', 'SP.POP.TOTL',
       'SP.URB.GROW', 'SP.URB.TOTL.IN.ZS', 'IncomeGroup', 'gdp'],
      dtype='object')

In [15]:
df_merged.to_csv('dashboard_data.csv', sep=';', decimal=',')